
## Imports

In [350]:
%reload_ext autoreload
%autoreload 2
from merkle_tree import MerkleTree
from channel import Channel
from hash import my_hash
from pprint import pprint

In [351]:
# Test

from pprint import pprint

m = MerkleTree([1, 2, 3, 4, 5, 6, 7, 8])
pprint([[b[:4] for b in a] for a in m.tree])
print(m.root)
pprint([a[:4] for a in m.get_path(0)])

[['6b86', 'd473', '4e07', '4b22', 'ef2d', 'e7f6', '7902', '2c62'],
 ['33b6', '1365', '4358', 'ada1'],
 ['85df', 'e0e2'],
 ['c274']]
c27450cd3fd4df029145f3437ae9c381e0ae55e8400de06cb973005b36d7b222
['6b86',
 ('d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35', 'right'),
 ('13656c83d841ea7de6ebf3a89e0038fea9526bd7f686f06f7a692343a8a32dca', 'right'),
 ('e0e2d0cec0ef7e8fc458e516dfde82890c183431a3f9efae9e4693fc23dfa36a', 'right')]


### Fibonacci function

In [352]:
def fibonacci(a=1, size=1001) -> list[int]:
    fib_list = [1, a]
    for _ in range(2, size):
        fib_list.append(fib_list[-1] + fib_list[-2])
    return fib_list

In [353]:
print(fibonacci())

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961, 4052739537881, 6557470319842, 10610209857723, 17167680177565, 27777890035288, 44945570212853, 72723460248141, 117669030460994, 190392490709135, 308061521170129, 498454011879264, 806515533049393, 1304969544928657, 2111485077978050, 3416454622906707, 5527939700884757, 8944394323791464, 14472334024676221, 23416728348467685, 37889062373143906, 61305790721611591, 99194853094755497, 160500643816367088, 259695496911122585, 420196140727489673, 679891637638612258, 110008777836

### Polynomial f(x)

In [354]:
P: int =3221225473
F = GF(P)
N = 8192

g = F(1734477367)
gamma = g ** 8
gamma_group = [gamma ** i for i in range(1001)]

fibonacci_seq = fibonacci()

R.<x> = PolynomialRing(F)

f = R.lagrange_polynomial(zip(gamma_group, fibonacci_seq))

### Bit Reverse Order

In [355]:
def bit_reverse(n, width):
    rev = 0  # This will store the reversed bits
    for i in range(width):
        rev = (rev << 1) | (n & 1)  # Shift left and take LSB of n
        n >>= 1  # Shift n to the right
    return rev


def bit_reverse_permutation(N):
    """Generate bit-reversed order for N elements."""
    width = int(math.log(N, 2))
    return [bit_reverse(i, width) for i in range(N)]


bit_reverse_order = bit_reverse_permutation(N)

### w < g >

In [356]:
w = F.multiplicative_generator()

g_group = [g ** i for i in range(N)]

wg = [w * gi for gi in g_group]
wg_merkle_order = [wg[i] for i in bit_reverse_order]

### Commit f(x) on LDE

In [357]:
channel = Channel(F, N)

f_wg = [f(x) for x in wg_merkle_order]

merkle_f_wg = MerkleTree(f_wg)

channel.send({'title': 'f(x) on LDE', 'data': merkle_f_wg.root})

### Polynomial h(x)

In [358]:
d_x = 1
for gamma_i in gamma_group[:-2]:
    d_x = d_x * (x - gamma_i)

F_x = f(x) + f(gamma * x) - f((gamma**2) * x)

h_x = F_x // d_x

In [359]:
h_wg = [F_x(x) / d_x(x) for x in wg_merkle_order]

# Sanity check
h_wg_by_poly = [h_x(x) for x in wg_merkle_order]
print(h_wg_by_poly[:10])
print(h_wg[:10])
assert(h_wg == h_wg_by_poly)

[1860826706, 1845649622, 290025805, 195225050, 2156032599, 1550443729, 1756057239, 1950419089, 356807959, 128442896]
[1860826706, 1845649622, 290025805, 195225050, 2156032599, 1550443729, 1756057239, 1950419089, 356807959, 128442896]


### Boundary Constraints

In [360]:
Y = fibonacci_seq[-1]

t0_x = (f(x) - 1) / (x - gamma ** 0)
t1_x = (f(x) - Y) / (x - gamma ** 1000)

In [361]:
t0_wg = [(f(i) - 1) / (i - gamma ** 0) for i in wg_merkle_order]
t1_wg = [(f(i) - Y) / (i - gamma ** 1000) for i in wg_merkle_order]

# Sanity check
t0_x_by_poly = [t0_x(x) for x in wg_merkle_order]
t1_x_by_poly = [t1_x(x) for x in wg_merkle_order]
print(t0_x_by_poly[:10])
print(t0_wg[:10])
print("")
print(t1_x_by_poly[:10])
print(t1_wg[:10])
assert(t0_wg == t0_x_by_poly)
assert(t1_wg == t1_x_by_poly)

[2384530853, 599226013, 2071111314, 1402510381, 2026908630, 951059204, 1906847992, 1092764146, 3083267015, 51241944]
[2384530853, 599226013, 2071111314, 1402510381, 2026908630, 951059204, 1906847992, 1092764146, 3083267015, 51241944]

[2395950328, 2696799910, 124207998, 3010185695, 1771367485, 510712279, 521057469, 2771188778, 1200373265, 956436872]
[2395950328, 2696799910, 124207998, 3010185695, 1771367485, 510712279, 521057469, 2771188778, 1200373265, 956436872]


### Composition Polynomial

In [362]:
beta_0 = channel.receive_random_field_element()
beta_1 = channel.receive_random_field_element()
beta_2 = channel.receive_random_field_element()

cp0_x = beta_0 * h_x + beta_1 * t0_x + beta_2 * t1_x

In [363]:
cp0_wg = [beta_0 * h_wg[i] + beta_1 * t0_wg[i] + beta_2 * t1_wg[i]
          for i in range(len(wg))]

# Sanity check
cp0_x_by_poly = [cp0_x(x) for x in wg_merkle_order]
print(cp0_x_by_poly[:10])
print(cp0_wg[:10])
assert(cp0_wg == cp0_x_by_poly)

[505489157, 2368666279, 1774153425, 2341526794, 1377743153, 1534405270, 1449131553, 978573396, 2713377995, 1641628493]
[505489157, 2368666279, 1774153425, 2341526794, 1377743153, 1534405270, 1449131553, 978573396, 2713377995, 1641628493]


### FRI

In [364]:
fri_layers = []
degree = R(cp0_x).degree()
# if degree % 2 != 0:
#     degree += 1

cp0_wg = [cp0_x(x) for x in wg_merkle_order]
merkle_cp_0 = MerkleTree(cp0_wg)
channel.send({
    "title": f"commit for cp 0",
    "data": merkle_cp_0.root,
})
z = channel.receive_random_field_element()
y_0 = f(z)
channel.send({
    "title": "y_0",
    "data": y_0,
})
y_1 = f(gamma * z)
channel.send({
    "title": "y_1",
    "data": y_1,
})
y_2 = f((gamma**2) * z)
channel.send({
    "title": "y_2",
    "data": y_2,
})
y_3 = cp0_x(z)
channel.send({
    "title": "y_3",
    "data": y_3,
})
alpha_0 = channel.receive_random_field_element()
alpha_1 = channel.receive_random_field_element()
alpha_2 = channel.receive_random_field_element()
alpha_3 = channel.receive_random_field_element()
fri_first_layer_poly = alpha_0 * (f - y_0) // (x - z) + alpha_1 * (f - y_1) // (x - gamma * z) + alpha_2 * (f - y_2) // (x - (gamma**2) * z) + alpha_3 * (cp0_x - y_3) // (x - z)
fri_first_layer_on_wg_by_poly = [fri_first_layer_poly(x) for x in wg_merkle_order]
curr_layer = [
    alpha_0 * (f(x) - y_0) / (x - z) + alpha_1 * (f(x) - y_1) / (x - gamma * z) + alpha_2 * (f(x) - y_2) / (x - (gamma**2) * z) + alpha_3 * (cp0_x(x) - y_3) / (x - z) for x in wg_merkle_order 
]
assert(curr_layer == fri_first_layer_on_wg_by_poly)

while degree > 0:
    # Commit curr layer
    merkle_curr_layer = MerkleTree(curr_layer)
    fri_layers.append((curr_layer, merkle_curr_layer))
    channel.send({
        "title": f"commit for layer {len(fri_layers) - 1} of FRI",
        "data": merkle_curr_layer.root,
    })
    random_x = channel.receive_random_field_element()
    curr_layer = [((curr_layer[i] + curr_layer[i + 1]) / 2) +
               random_x * (curr_layer[i] - curr_layer[i + 1]) / (2 * wg_merkle_order[i]) for i in range(len(curr_layer))[::2]]
    degree = degree // 2

constant = curr_layer[0]
channel.send({"title:": "constant of last FRI layer",
              "data": constant,
              })

assert(all([constant == x for x in curr_layer]))

In [365]:
print(fri_layers)
print("Merkle Root")
print(curr_layer[0])

[([452722249, 1672321766, 1168332673, 681879295, 3113907893, 2471625984, 3111946506, 259940911, 2982522397, 1891839178, 778174866, 1703798785, 897197092, 1886773827, 2746074593, 748707512, 2228273470, 1122282077, 1275991831, 2486991297, 1511358786, 1601714981, 1459477343, 2987825403, 2403731468, 2833125166, 1891125485, 2916680257, 1248702725, 2935616675, 2482418951, 2834010273, 2410705733, 2274227181, 2016833792, 2498815214, 1872249651, 2007619006, 2801097593, 1809527617, 782243150, 1167421557, 1266981559, 5512073, 591693757, 1527822869, 1207234561, 861549874, 2487966896, 2742561380, 709517038, 95127207, 1125548017, 2544173936, 421851925, 2112009561, 2348058505, 1775286142, 2356982471, 3097065763, 687697632, 2720763305, 2793186587, 510286537, 1497581216, 1146559118, 146260181, 581341853, 1314070673, 2831393828, 104432717, 2388887342, 2733910435, 2854275681, 2335443355, 1750997575, 870373763, 407708323, 228575665, 2001957152, 1571360669, 105552589, 1385157961, 1246359124, 981364669, 709

In [366]:
# Sanity check
print(curr_layer)

[2488569551, 2488569551, 2488569551, 2488569551, 2488569551, 2488569551, 2488569551, 2488569551]


In [367]:
pprint(channel.get_all_messages())

[{'data': 'd524fba68e1263f084afe501c715dca773c616d5a8c52303322985d32c94ea56',
  'title': 'f(x) on LDE'},
 {'data': 2415762890, 'title': 'Random Field Element'},
 {'data': 1173033395, 'title': 'Random Field Element'},
 {'data': 519019791, 'title': 'Random Field Element'},
 {'data': '1a10f17d9c3fd15f47317a3a8620d4faabd3e7a2010a9a7f5e7905b67f417c3b',
  'title': 'commit for cp 0'},
 {'data': 3147769716, 'title': 'Random Field Element'},
 {'data': 315097056, 'title': 'y_0'},
 {'data': 3218501028, 'title': 'y_1'},
 {'data': 1345121225, 'title': 'y_2'},
 {'data': 2972751031, 'title': 'y_3'},
 {'data': 1900694309, 'title': 'Random Field Element'},
 {'data': 106282677, 'title': 'Random Field Element'},
 {'data': 2597370735, 'title': 'Random Field Element'},
 {'data': 175232189, 'title': 'Random Field Element'},
 {'data': 'ae7751b91f84954b55be1b232df26d84825ae64d3ce92e9287cc0a46de96af16',
  'title': 'commit for layer 0 of FRI'},
 {'data': 1700653970, 'title': 'Random Field Element'},
 {'data': '

### Decommit Phase

In [368]:
def send_f_of_x_and_cp_of_x(idx_of_x):
    channel.send({
        'title': 'f of x',
        'data': f_wg[idx_of_x],
    })
    channel.send({
        'title': 'path of f of x',
        'data': merkle_f_wg.get_path(idx_of_x),
    })
    channel.send({
        'title': 'cp of x',
        'data': cp0_wg[idx_of_x],
    })
    channel.send({
        'title': 'path of cp of x',
        'data': merkle_cp_0.get_path(idx_of_x),
    })

def send_cp_layer(idx_of_x, idx_of_minus_x, cp, cp_merkle, i):
    assert(cp_merkle.get_path(idx_of_x)[2:] == cp_merkle.get_path(idx_of_minus_x)[2:])
    channel.send({
        'title': f'path of cp of x - layer {i}',
        'data': cp_merkle.get_path(idx_of_x),
    })
    channel.send({
        'title': f'cp of -x - layer {i}',
        'data': cp[idx_of_minus_x],
    })
    channel.send({
        'title': f'path of cp of -x - layer {i}',
        'data': cp_merkle.get_path(idx_of_minus_x),
    })

def query(idx):
    idx_of_x = bit_reverse_order.index(idx)
    send_f_of_x_and_cp_of_x(idx_of_x)
    for i, (layer, layer_merkle) in enumerate(fri_layers):
        idx_of_minus_x = idx_of_x ^^ 1
        send_cp_layer(idx_of_x, idx_of_minus_x, layer, layer_merkle, i)
        idx_of_x //= 2

In [369]:
channel.start_decommit()
for i in range(8):
    random_idx = channel.receive_random_query()
    query(random_idx)

In [370]:
def check_merkle_path(data, path, root):
    curr = my_hash(data)
    if curr != path[0]:
        return False
    for sibling, direction in path[1:]:
        if direction == 'left':
            curr = my_hash(sibling + curr)
        else:
            curr = my_hash(curr + sibling)
    return curr == root

# Sanity
assert(check_merkle_path(f_wg[4000], merkle_f_wg.get_path(4000), merkle_f_wg.root))

# Verifier

In [371]:
channel.receive_decommitment()

[{'title': 'Random Query', 'data': 2402},
 {'title': 'f of x', 'data': 1127834752},
 {'title': 'path of f of x',
  'data': ['835c86fe93b643b4be3327e77e7bac6c14c86f56e491a5658b6a025c88e8febb',
   ('60b803eda072a5ccc94aeedfce0bd3398cb7925015682a75d50e58e24e76cdb2',
    'right'),
   ('190b9f9c28268cdad206a71e8905eaffc49e66bea0146d8f01765f9dc0985bb0',
    'left'),
   ('49f988fcec3e4d4e92721c06fdc1e7fd6f1da6126c177cab2e19e53365dd8df4',
    'right'),
   ('f130b3b77d6c313faf0ce38d94840e90485a0dc5a7ec2fc5ea800c46be70da82',
    'right'),
   ('5d9f0733f601d2efdcc4c4db59575659afdc7367cfdf3350a2d44b38c94cb1f4',
    'left'),
   ('44e804e5a22b2083247ff566ddcf0885d0e3816cf1e4ab21b98245b0c893b314',
    'right'),
   ('35c1d7242886c818412ad2824370bf9851976537600c10d0178de56fac8bfa0a',
    'left'),
   ('345dcd670def73702a8d51ffd4861506255c55b0460eddd7c520cd6afd8935d6',
    'left'),
   ('904fed1770dc39f5a81f76e7b3892792d3ff35975f3a016a9d79c904c12e7889',
    'right'),
   ('b1971e1937445e93e14db93b158dcf581

In [372]:
P: int = 3221225473
F = GF(P)
N = 8192
g = F(1734477367)
gamma = g ** 8
Y = fibonacci_seq[-1]

# Initial tree
commitment = channel.receive_commitment()
decommitment = channel.receive_decommitment()
random_idx = decommitment[0]['data']
# f_x
f_x = decommitment[1]['data']
path_of_f_x = decommitment[2]['data']
root_f_x = commitment[0]['data']
assert(check_merkle_path(f_x, path_of_f_x, root_f_x))
# cp 0 x
fri_layer = decommitment[3]['data']
path_of_cp_x = decommitment[4]['data']

beta_0 = commitment[1]['data']
beta_1 = commitment[2]['data']
beta_2 = commitment[3]['data']

root_cp_x = commitment[4]['data']
assert(check_merkle_path(fri_layer, path_of_cp_x, root_cp_x))

z = commitment[5]['data']
f_z = commitment[6]['data']
f_gamma_z = commitment[7]['data']
f_gamma_2_z = commitment[8]['data']
y_3 = commitment[9]['data']

F_z = f_z + f_gamma_z - f_gamma_2_z

d_z_denominator = 1
for gamma_i in [gamma ** i for i in range(999, 1024)]:
    d_z_denominator = d_z_denominator * (z - gamma_i)
d_z = (z ** 1024 - 1) / d_z_denominator

h_z = F_z / d_z
t0_z = (f_z - 1) / (z - gamma ** 0)
t1_z = (f_z - Y) / (z - gamma ** 1000)
cp_z = beta_0 * h_z + beta_1 * t0_z + beta_2 * t1_z

assert(cp_z == y_3)

alpha_0 = commitment[10]['data']
alpha_1 = commitment[11]['data']
alpha_2 = commitment[12]['data']
alpha_3 = commitment[13]['data']

g_pow = g ** random_idx
query_x = w * g_pow

fri_layer = alpha_0 * (f_x - f_z) / (query_x - z) + alpha_1 * (f_x - f_gamma_z) / (query_x - gamma * z) + \
    alpha_2 * (f_x - f_gamma_2_z) / (query_x - (gamma ** 2) * z) + alpha_3 * (fri_layer - cp_z) / (query_x - z)

commitment_idx = 14
decommitment_idx = 5
while commitment_idx < len(commitment) - 1:
    path_of_cp_x = decommitment[decommitment_idx]['data']
    cp_minus_x = decommitment[decommitment_idx + 1]['data']
    path_of_cp_minus_x = decommitment[decommitment_idx + 2]['data']

    cp_root = commitment[commitment_idx]['data']
    assert(check_merkle_path(fri_layer, path_of_cp_x, cp_root)), (fri_layer, path_of_cp_x, cp_root)
    assert(check_merkle_path(cp_minus_x, path_of_cp_minus_x, cp_root))

    random_x = commitment[commitment_idx + 1]['data']
    query_x = w * g_pow
    fri_layer = (fri_layer + cp_minus_x) / 2 + random_x * (fri_layer - cp_minus_x) / (2 * query_x)
    
    g_pow = g_pow ** 2

    commitment_idx += 2
    decommitment_idx += 3

assert(fri_layer == commitment[commitment_idx]['data'])